# Chapter 15 — Deployment & Applications (Colab)

Export a bundle, validate it, load it, and sample. These cells mirror the
chapter’s Streamlit/FastAPI wiring so you can sanity‑check a bundle inside
a notebook before shipping it.


In [ ]:
# Bundle structure (config + weights + tokenizer)
import json, torch
example = {
  'config': {'vocab_size': 32, 'block_size': 8, 'd_model': 32,
             'n_head': 4, 'n_layer': 2, 'd_ff': 64, 'dropout': 0.0},
  'model_state': {},
  'tokenizer': {'level': 'char', 'id_to_token': list(' _abcdefghijklmnopqrstuvwxyz'),
                'pad_id': 0, 'unk_id': 1}
}
list(example.keys())


In [ ]:
# Load + sample from a real bundle (requires repo present and a bundle path)
import sys, pathlib
sys.path.append(str(pathlib.Path('code').resolve()))
from ch09_gpt import GPT, GPTConfig  # type: ignore
from ch11_sampling import sample      # type: ignore
from ch6_tokenize import SimpleTokenizer, Vocab  # type: ignore

def load_bundle(path: str):
    b = torch.load(path, map_location='cpu')
    cfg = GPTConfig(**b['config'])
    model = GPT(cfg).eval(); model.load_state_dict(b['model_state'])
    meta = b.get('tokenizer'); tok = None
    if meta and meta.get('id_to_token'):
        id_to_token = list(meta['id_to_token'])
        token_to_id = {t:i for i,t in enumerate(id_to_token)}
        vocab = Vocab(token_to_id=token_to_id, id_to_token=id_to_token,
                     pad=int(meta.get('pad_id',0)), unk=int(meta.get('unk_id',1)))
        tok = SimpleTokenizer(vocab=vocab, level=meta.get('level','char'))
    return model, tok

def sample_bundle(bundle_path: str, prompt: str,
                  max_new=80, temperature=0.9, top_p=0.95, top_k=0):
    model, tok = load_bundle(bundle_path)
    if tok is None:
        ids = torch.tensor([[c for c in prompt.encode('utf-8')]], dtype=torch.long)
        out = sample(model, ids, max_new_tokens=max_new, temperature=temperature,
                     top_k=(top_k or None), top_p=(top_p or None))
        return bytes(out[0].tolist()).decode('utf-8', errors='ignore')
    ids = torch.tensor([tok.encode(prompt)], dtype=torch.long)
    out = sample(model, ids, max_new_tokens=max_new, temperature=temperature,
                 top_k=(top_k or None), top_p=(top_p or None))
    return tok.decode(out[0].tolist())

# Example (requires a real bundle):
# sample_bundle('model_bundle.pt', 'Hello')
